### AUDIOVAULT: IMAGE IN AUDIO STEGANOGRAPHY

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from PIL import Image
from utils import image_to_bits, check_bits_alteration
from embedding import embed_data
from extraction import extract_data
from config import TWO_BIT_QIM

In [3]:
#FILE PATHS
audio_path = "audio/orcestral1.wav"
stego_path = "stego.wav"
secret_data_path = "images/secret.jpg"
extracted_data_path = "extracted.png"

In [4]:
secret_data, secret_shape = image_to_bits(path=secret_data_path)
files = [
    {
        "file_type":"jpg",
        "height":secret_shape[0],
        "width":secret_shape[1],
        "channels":secret_shape[2],
        "payload_length":len(secret_data)
    }
]

In [5]:
#Embed the secret data
embed_data(secret_data=secret_data,files=files,audio_path=audio_path,stego_path=stego_path)

HEADER BITS: ['0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0']
DECODED HEADER: {'version': 1, 'header_length': 112, 'file_count': 1, 'two_bit_qim': 1, 'files': [{'file_type': 'jpg', 'height': 427, 'width': 640, 'channels': 3, 'payload_length': 6558720}]}
Frame Size: 1770910
TOTAL FRAMES: 5
Payload Capacity: 3320414 and Payload Length: 3279360
Payload Capacity: 3320414 and Payload Length: 3279360
Payload Capacity: 3320414 and Payload Length: 3279360
Payload Capacity: 3320414 and Payloa

In [ ]:
#Extract the secret data
extracted_data = extract_data(audio_path="uploads/song_cropped.wav",extracted_data_path=extracted_data_path)

Sync Bits Matched at index: 7
Frame Size Extracted as: 1770910
------CHECKING HEADER BITS - EXTRACTION----------
['0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0']
------END OF CHECKING HEADER BITS - EXTRACTION----------
------CHECKING DECODED HEADER - EXTRACTION----------
{'version': 1, 'header_length': 112, 'file_count': 1, 'two_bit_qim': 1, 'files': [{'file_type': 'jpg', 'height': 427, 'width': 640, 'channels': 3, 'payload_length': 6558720}]}
------END OF CHECKING DECODED HEADER - E

In [7]:
check_bits_alteration(original=secret_data, extracted=extracted_data)

Mismatch location: 69960
Mismatch location: 218527
Mismatch location: 218528
Mismatch location: 218532
Mismatch location: 218537
Mismatch location: 218538
Mismatch location: 218543
Mismatch location: 218545
Mismatch location: 218546
Mismatch location: 218547
Mismatch location: 218548
Mismatch location: 218550
Mismatch location: 218551
Mismatch location: 218553
Mismatch location: 218555
Mismatch location: 218559
Mismatch location: 218560
Mismatch location: 218561
Mismatch location: 218562
Mismatch location: 218563
Mismatch location: 218564
Mismatch location: 218566
Mismatch location: 218567
Mismatch location: 218569
Mismatch location: 218571
Mismatch location: 218572
Mismatch location: 218574
Mismatch location: 218577
Mismatch location: 218579
Mismatch location: 218580
Mismatch location: 218581
Mismatch location: 218582
Mismatch location: 218584
Mismatch location: 218588
Mismatch location: 218589
Mismatch location: 218592
Mismatch location: 218593
Mismatch location: 218594
Mismatch loca